In [13]:
!pip install streamlit pyngrok==4.1.1
!pip install streamlit torch torchvision pillow

In [11]:
from google.colab import files
uploaded = files.upload()

Saving malaria_resnet50_best.pth to malaria_resnet50_best.pth


In [7]:
from pyngrok import ngrok
ngrok.set_auth_token("35PlaCE8Cifp909UllhveAHKCXk_435egM8UYKBLfYFggEN6r")

In [9]:
%%writefile malaria_app.py
import streamlit as st
from PIL import Image
import torch
from torch import nn
from torchvision import transforms, models

# -----------------------------
# Title & description
# -----------------------------
st.title(" Malaria Detection App (CNN + ResNet50)")
st.write("Upload a blood cell image to classify whether it is **Parasitized** or **Uninfected**.")

# -----------------------------
# Class names
# -----------------------------
class_names = ["Parasitized", "Uninfected"]

# -----------------------------
# Device
# -----------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# -----------------------------
# Build the SAME model as in training
# -----------------------------
# No pretrained weights needed here, we load our own trained weights
base_model = models.resnet50(weights=None)
in_features = base_model.fc.in_features
base_model.fc = nn.Linear(in_features, 2)

# Load trained weights
STATE_PATH = "malaria_resnet50_best.pth"

try:
    state_dict = torch.load(STATE_PATH, map_location=device)
    base_model.load_state_dict(state_dict)
except FileNotFoundError:
    st.error(f"Model weights file '{STATE_PATH}' not found. "
             "Make sure it is in the same folder as this script.")
    st.stop()

model = base_model.to(device)
model.eval()

# -----------------------------
# Preprocessing
# -----------------------------
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
    ),
])

def load_input_image(pil_img: Image.Image) -> torch.Tensor:
    """Convert a PIL image to a normalized tensor with batch dimension."""
    img = pil_img.convert("RGB")
    tensor = preprocess(img).unsqueeze(0)  # [1, C, H, W]
    return tensor.to(device)

# -----------------------------
# Streamlit UI
# -----------------------------
uploaded_file = st.file_uploader(
    "Upload a cell image (.png, .jpg, .jpeg)",
    type=["png", "jpg", "jpeg"]
)

if uploaded_file is not None:
    # Show image
    image = Image.open(uploaded_file)
    st.image(image, caption="Uploaded Image", width=250)

    if st.button("Predict"):
        # Preprocess and predict
        with torch.no_grad():
            x = load_input_image(image)
            logits = model(x)
            probs = torch.softmax(logits, dim=1)[0]
            pred_idx = int(torch.argmax(probs))
            pred_class = class_names[pred_idx]
            confidence = float(probs[pred_idx])

        st.subheader("Prediction")
        st.write(f"**Class:** {pred_class}")
        st.write(f"**Confidence:** {confidence:.3f}")

Overwriting malaria_app.py


In [14]:
!ls -lh

total 110M
-rw-r--r-- 1 root root 1.1K Nov 13 08:04 app.py
-rw-r--r-- 1 root root  20M Nov  7 17:06 cloudflared-linux-amd64.deb
-rw-r--r-- 1 root root 2.7K Nov 13 18:35 malaria_app.py
-rw-r--r-- 1 root root  90M Nov 13 18:49 malaria_resnet50_best.pth
drwxr-xr-x 1 root root 4.0K Nov 11 14:29 sample_data


In [17]:
!nohup streamlit run malaria_app.py --server.port 8501 --server.address 0.0.0.0 > /dev/null 2>&1 &

In [18]:
from pyngrok import ngrok

public_url = ngrok.connect(8501)
public_url

ERROR:pyngrok.process:t=2025-11-13T18:59:10+0000 lvl=eror msg="failed to auth" obj=tunnels.session err="Your ngrok-agent version \"2.3.41\" is too old. The minimum supported agent version for your account is \"3.7.0\". Update to a newer version with 'ngrok update', by downloading from https://ngrok.com/download, or by updating your SDK version. Paid accounts are currently excluded from minimum agent version requirements. To begin handling traffic immediately without updating your agent, upgrade to a paid plan: https://dashboard.ngrok.com/billing/choose-a-plan.\r\n\r\nERR_NGROK_121\r\n"



PyngrokNgrokError: The ngrok process errored on start: Your ngrok-agent version "2.3.41" is too old. The minimum supported agent version for your account is "3.7.0". Update to a newer version with 'ngrok update', by downloading from https://ngrok.com/download, or by updating your SDK version. Paid accounts are currently excluded from minimum agent version requirements. To begin handling traffic immediately without updating your agent, upgrade to a paid plan: https://dashboard.ngrok.com/billing/choose-a-plan.\r\n\r\nERR_NGROK_121\r\n.